# YOLOv8

## Instalação e Configuração

In [ ]:
import os
import sys

helpers_path = os.path.abspath("..")

sys.path.append(helpers_path)

# Remover diretorios runs/
import shutil
if os.path.exists('runs/'):
    shutil.rmtree('runs/')

In [ ]:
# !pip install ultralytics

In [ ]:
import ultralytics

In [ ]:
ultralytics.checks()

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
# conda install pytorch torchvision torchaudio pytorch-cuda=11.7 -c pytorch -c nvidia

In [ ]:
!nvidia-smi

### Clonar repositório (alternativa)

In [ ]:
#!git clone https://github.com/ultralytics/ultralytics
#!pip install -r /content/ultralytics/requirements.txt
#!pip install -qe ultralytics

### Importação das demais bibliotecas

In [ ]:
import cv2
import matplotlib.pyplot as plt
import os
from PIL import Image

## Download do modelo

- Os modelos do YOLOv8 podem ser baixados aqui:
https://github.com/ultralytics/assets/releases

In [ ]:
# baixar o modelo e salve na pasta shared
#!wget https://github.com/ultralytics/assets/releases/download/v0.0.0/yolov8n.pt

## Realizando a detecção

A forma mais rápida de executar a deteçcão é através da CLI, conforme pode ver abaixo a sintaxe.

Os parâmetros:
* **task** - tarefa, podendo ser detecção, segmentação ou classificação
* **model** - o modelo que queremos usar. São disponibilizados 5 modelos: YOLOv8n (nano), YOLOv8s (small), YOLOv8m (medium), YOLOv8l (large), YOLOv8x (extra large). Nano é o mais rápido e o menor (menos pesado para rodar), enquanto que o Extra Large (YOLOv8x) é o mais preciso porém mais pesado para rodar, portanto será mais lento.
  Para o valor desse parâmetro, basta informar o nome e ao lado .pt pois é a extensão do modelo treinado em pytorch.

* **mode** - basicamente é o que queremos com o comando. queremos fazer a detecção/inferência/predição, portanto deixe =predict. Valores aceitos: [train, val, predict, export]

* **conf** - o limiar (threshold) que usaremos para filtrar detecções "fracas". Se a confiança estiver abaixo desse valor, não será considerada. Por padrão podemos configurar com um limiar bem baixo (ex: 0.25) e depois aumentar, caso verifique que detectou incorretamente algum objeto.  
* **source** - a imagem ou vídeo que queremos fazer a detecção.

In [ ]:
# Detectar objetos em uma imagem local
# !yolo task=detect mode=predict model=yolov8n.pt conf=0.25 source='/content/cachorros.jpg' save=True

# Ou execute:

from helpers.yolo import detectar_objetos
from helpers.opencv import carregar_imagem

detectar_objetos('cachorros.jpg')

carregar_imagem('runs/detect/predict/cachorros.jpg', True)

In [ ]:
# Detectar objetos em uma imagem via URL
# !yolo task=detect mode=predict model=yolov8n.pt conf=0.25 source='https://ultralytics.com/images/zidane.jpg' save=True

# Ou execute:

from helpers.yolo import detectar_objetos_url

detectar_objetos_url('https://ultralytics.com/images/zidane.jpg')

carregar_imagem('runs/detect/predict2/zidane.jpg', True)

### Alterando o limiar (threshold)

O *threshold* é controlado pelo parâmetro **conf**. O valor padrão é 0.25, ou seja, todas as detecções com confiança abaixo de 25% serão descartadas.

Aumentá-lo pode ajudar a reduzir falsos positivos, mas também pode fazer com que alguns objetos reais não sejam detectados.

Diminuir o limiar pode aumentar a sensibilidade, mas também pode aumentar o número de falsos positivos.

In [ ]:
# Alterando o limiar (threshold) para 0.4
# !yolo task=detect mode=predict model=yolov8n.pt conf=0.4 source='/content/italia.jpg' save=true

# Ou execute:

detectar_objetos('italia.jpg', conf=0.4)

carregar_imagem('runs/detect/predict3/italia.jpg', True)

In [ ]:
# Alterando o limiar (threshold) para 0.6
# !yolo task=detect mode=predict model=yolov8n.pt conf=0.6 source='/content/italia.jpg' save=true

# Ou execute:

detectar_objetos('italia.jpg', conf=0.6)

carregar_imagem('runs/detect/predict4/italia.jpg', True)

## Detecção em múltiplas imagens de uma vez

Coloque todas as imagens em uma pasta e informe o caminho da pasta no parâmetro **source**. O YOLOv8 irá processar todas as imagens da pasta de uma vez só.

In [ ]:
# Detecção em múltiplas imagens de uma vez
# !yolo task=detect mode=predict model=yolov8n.pt conf=0.5 source='/content/imagens' save=true

# Ou execute:
from helpers.yolo import detectar_objetos_imagens
from helpers.settings import IMG_PATH

detectar_objetos_imagens(IMG_PATH, conf=0.5)

In [ ]:
dir_resultados = 'runs/detect/predict5/'

In [ ]:
caminhos = [os.path.join(dir_resultados, f) for f in os.listdir(dir_resultados)]
caminhos

In [ ]:
for caminho_imagem in caminhos:
    carregar_imagem(caminho_imagem, True)

## Realizando a detecção - método através do Python

In [ ]:
from ultralytics import YOLO

In [ ]:
model = YOLO('yolov8n.pt')

In [ ]:
from helpers.opencv import mostrar_imagem

img = carregar_imagem('cachorros.jpg', False)

mostrar_imagem(img)

Para a deteção usaremos o método model.predict().

Ele aceita basicamente os mesmos parâmetros do método via CLI, que vimos nos exemplos mais acima.

* source -- A função aceita a imagem nos seguintes formatos:
  * imagem em ndarray (usando o cv2.imread) ou PIL
  * diretório
  * caminho
  * URL
  * video
  * webcam (valor 0, que corresponde ao indice do dispositivo de câmera)

* save -- se desejar salvar a imagem no diretório predict deixei save=True. aqui nós vamos fazer de outra maneira, então não precisa setar esse parâmetro

* conf -- limiar de confiança

* show -- Informe o parâmetro show=True para mostrar os valores das predições

* save_txt -- se deseja salvar em um txt os resultados

mais parâmetros https://docs.ultralytics.com/cfg/

In [ ]:
resultados = model.predict(source=img)

In [ ]:
resultados

In [ ]:
for r in resultados:
  print(r.boxes.data)  # caixas delimitadoras (bounding boxes)

In [ ]:
for r in resultados:
  print(r.boxes.conf) # confianças (confidence scores) de cada caixa

### Desenhando a caixa delimitadora sobre a imagem



In [ ]:
from helpers.opencv import desenha_caixas

resultado = desenha_caixas(img, resultados[0].boxes.data)
mostrar_imagem(resultado)

## Teste com outros modelos


> Usando o YOLOv8m (medium)

In [ ]:
model = YOLO('yolov8m.pt')

img = carregar_imagem('italia.jpg', False)

resultados = model.predict(source = img, conf=0.6)
resultado_img = desenha_caixas(img, resultados[0].boxes.data)

mostrar_imagem(resultado_img)

> Usando o YOLOv8x (extra large)

In [ ]:
model = YOLO('yolov8x.pt')

img = carregar_imagem('italia.jpg', False)

resultados = model.predict(source = img, conf=0.6)
resultado_img = desenha_caixas(img, resultados[0].boxes.data)

mostrar_imagem(resultado_img)

## Validação do modelo

As principais métricas de avaliação são mAP50 e mAP50-95
- mAP50: Mean Average Precision para um IoU threshold de 0.5
- mAP50-95: Mean Average Precision para IoU thresholds variando de 0.5 a 0.95 em incrementos de 0.05
Em que IoU é Intersection over Union, que mede a sobreposição entre a caixa delimitadora prevista e a caixa delimitadora real. E IoU threshold é o valor mínimo de IoU necessário para considerar uma detecção como verdadeira positiva.

In [ ]:
# Download de imagens do dataset COCO para validação do modelo treinado
import torch
torch.hub.download_url_to_file('https://ultralytics.com/assets/coco2017val.zip', 'tmp.zip')

In [ ]:
!rm -rf ../datasets
!unzip -q tmp.zip -d ../datasets && rm tmp.zip

Abaixo estão os comandos para validar os modelos YOLOv8x, YOLOv8m e YOLOv8n usando o dataset coco128.yaml, que é um subconjunto do COCO com 128 imagens. Observe como o mAP50 e mAP50-95 de cada classe variam conforme o modelo.

In [ ]:
# Validação do modelo yolov8x.pt
!yolo task=detect mode=val model=yolov8x.pt data=coco128.yaml

In [ ]:
# Validação do modelo yolov8m.pt
!yolo task=detect mode=val model=yolov8m.pt data=coco128.yaml

In [ ]:
# Validação do modelo yolov8n.pt
!yolo task=detect mode=val model=yolov8n.pt data=coco128.yaml

## Detecção em vídeos

In [ ]:
# Detecção em vídeo
# !yolo task=detect mode=predict model=yolov8m.pt conf=0.5 source='/content/video_pessoas01.mp4' save=true

# Ou execute:

from helpers.settings import VIDEO_PATH
from helpers.yolo import detectar_objetos_video

detectar_objetos_video(f'{VIDEO_PATH}/video_rua01.mp4', conf=0.5)


In [ ]:
from helpers.settings import VIDEO_PATH

import os

video_salvo = f'runs/detect/predict6/video_rua01.avi'
video_final = f'{VIDEO_PATH}/video_rua01_resultado.mp4'
os.system(f"ffmpeg -i {video_salvo} -vcodec libx264 {video_final}")

In [ ]:
from helpers.opencv import exibir_video

exibir_video(video_final)

## Segmentação de imagens

Os modelos de segmentação do YOLOv8 terminam com "-seg", por exemplo: yolov8m-seg.pt

### Via CLI

In [ ]:
# Segmentação de imagens
# !yolo task=segment mode=predict model=yolov8m-seg.pt conf=0.5 source='/content/imagens/cachorros.jpg' save=true

# Ou execute:
# TODO: implementar função de segmentação no helpers/yolo.py

In [ ]:
# TODO: exibir imagem segmentada

### Via Python

In [ ]:
model = YOLO('yolov8m-seg.pt')

img = carregar_imagem('person.jpg', False)

resultados = model.predict(source = img, conf=0.6)
resultado_img = desenha_caixas(img, resultados[0].boxes.data)

mostrar_imagem(resultado_img)

In [ ]:
model = YOLO('yolov8x-seg.pt')

img = carregar_imagem('person.jpg', False)

resultados = model.predict(source = img, conf=0.6)
resultado_img = desenha_caixas(img, resultados[0].boxes.data)

mostrar_imagem(resultado_img)

### Segmentação em vídeos

In [ ]:
# !wget https://github.com/gabevr/yolo/raw/master/videos/video_rua01.mp4

In [ ]:
# Segmentação em vídeo
# !yolo task=segment mode=predict model=yolov8m-seg.pt conf=0.5 source='/content/video_rua01.mp4' save=true

# Ou execute:
# TODO implementar função de segmentação em vídeo no helpers/yolo.py

In [ ]:
video_salvo = "runs/segment/predict7/video_rua01.mp4"
video_final = f"{VIDEO_PATH}/segmentacao_rua01.mp4"

# os.system(f"ffmpeg -i {video_salvo} -vcodec libx264 {video_final}")

In [ ]:
# exibir_video(video_final)